<a href="https://colab.research.google.com/github/Zeeshan506/developerhub-task-1-m2-news-classifier/blob/main/Gradio_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import HfApi
from google.colab import userdata

HF_WRITE_TOKEN = userdata.get('HF_WRITE_TOKEN')

repo_id = "Zeeshan506/news-classifier"
subfolder = "checkpoint-13500"

# ✅ Load model weights from checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    repo_id,
    subfolder=subfolder,
    token=HF_WRITE_TOKEN
)

# ✅ Load tokenizer from base model (since checkpoint has no tokenizer files)
tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-uncased"
)

# ✅ Save clean final model
save_dir = "./news-classifier-final"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# ✅ Upload final model to repo root
api = HfApi()
api.upload_folder(
    folder_path=save_dir,
    repo_id=repo_id,
    repo_type="model",
    token=HF_WRITE_TOKEN,
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...-classifier-final/model.safetensors:   8%|7         | 33.5MB /  438MB            

CommitInfo(commit_url='https://huggingface.co/Zeeshan506/news-classifier/commit/1897e00c88a001f4dacbd609e1151e77701877fa', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1897e00c88a001f4dacbd609e1151e77701877fa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Zeeshan506/news-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='Zeeshan506/news-classifier'), pr_revision=None, pr_num=None)

In [12]:
# Install Gradio
!pip install gradio -q

import gradio as gr
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load model and tokenizer
repo_id = "Zeeshan506/news-classifier"
model = AutoModelForSequenceClassification.from_pretrained(repo_id)
tokenizer = AutoTokenizer.from_pretrained(repo_id)
model.eval()

# Manually create the label mapping
# This corresponds to the AG News dataset labels
ID_TO_LABEL = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}

# Classification function
def classify_text(text):
    if not text:
        return {"None": 0.0}
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the raw scores from the logits
    scores = torch.softmax(outputs.logits, dim=1)[0].tolist()

    # Map the scores to the correct labels using the dictionary
    predictions = {ID_TO_LABEL[i]: scores[i] for i in range(len(scores))}

    return predictions

# Create the Gradio interface
demo = gr.Interface(
    fn=classify_text,
    inputs=gr.Textbox(lines=2, placeholder="Enter news headline here..."),
    outputs=gr.Label(),
    title="News Topic Classifier",
    description="Classify a news headline into categories like World, Sports, Business, or Sci/Tech.",
    examples=[
        ["Apple unveils new AI-powered iPhone with improved camera"],
        ["Local team wins championship with last-second shot"],
        ["Stock market soars as inflation fears ease"]
    ]
)

# Launch the demo with a public URL
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12455162c1d8801075.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
